<a href="https://colab.research.google.com/github/ev1025/happy_dog_map/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%A7%80%EB%8F%84_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import math
import re
import pandas as pd
import numpy as np
import csv

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 크롤링 데이터 리스트
crawl_list = []

# 크롬드라이버
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 네이버지도 방문
driver.get('https://map.naver.com/v5/?c=15,0,0,0,dh')

# 네이버지도 페이지 로드시간 대기
driver.implicitly_wait(10)

# 검색어 목록
search_list = ['서울 애견 미용실' ,'서울 애견호텔', '서울 반려동물 미용실', '서울 동물약국',\
                  '서울 동물병원', '서울 강아지 수제간식','서울 애견 수제간식',  '서울 강아지유치원','서울 애견유치원', '서울 애견용품']

for search_id in search_list:
    # 검색상자 찾기
    search_box = driver.find_element(By.CSS_SELECTOR, 'input.input_search')
    # 검색어 입력
    search_box.send_keys(f'{search_id}')
    # 검색버튼 클릭
    search_box.send_keys(Keys.ENTER)
    # 검색결과창(iframe)으로 셀레니움 이동
    driver.switch_to.frame('searchIframe')
    # 데이터 담을 목록
    

    # 페이지 수 만큼 반복
    for page in range(30):
        # ======= 스크롤 시작 ========
        # 스크롤키를 적용하기 위해 해당 iframe창 클릭
        driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container").click()
        # 초기 검색된 개수
        lis = driver.find_elements(By.CSS_SELECTOR, "li.VLTHu.OW9LQ")
        before_len = len(lis)
        while True:
            # 스크롤 다운 1회
            driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
            time.sleep(1.5)

            # 스크롤 다운 후 검색된 개수
            lis = driver.find_elements(By.CSS_SELECTOR, "li.VLTHu.OW9LQ")
            after_len = len(lis)

            # 초기 개수와 스크롤 후 개수가 같아지면 더 이상 갱신값이 없다는 뜻
            if before_len == after_len:
                break
            # 다음 비교를 위해 갱신
            before_len = after_len
        # ======= 스크롤 끝 =======

        # 검색결과 모음
        button_box = driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink.YwYLL')

        # 검색결과 1개씩 눌러서 크롤링
        for button in button_box:
            time.sleep(2.5)
            button.click()

            # 검색 상세창으로 iframe이동
            driver.implicitly_wait(5)    
            driver.switch_to.default_content()
            driver.switch_to.frame('entryIframe')

            # 상세정보 가져오기
            maplayout = "document.querySelector('#app-root > div > div > div').innerHTML"
            page_raw = driver.execute_script("return " + maplayout)
            page_html = BeautifulSoup(page_raw, 'html.parser')
            
            # 데이터(상호명, 업종, 리뷰 수, 별점, 주소, 전화번호)
            temp = []
            name = page_html.select_one('span.Fc1rA').text
            category = page_html.select_one('span.DJJvD').text
            try: 
                vis_review = int(page_html.select_one('div.dAsGb > span:nth-child(1) > a > em').text)
            except:
                vis_review = 0
            try:
                blog_review = int(page_html.select_one('div.dAsGb > span:nth-child(2) > a > em').text)
            except:
                blog_review = 0
            review_num = vis_review + blog_review
            try:
                star = int(page_html.select_one('div.dAsGb > span.PXMot.LXIwF > em').text)
            except:
                star = np.NaN
            adr = page_html.select_one('span.LDgIH').text.split()
            address = adr[1]
            try:
                tel = page_html.select_one('span.xlx7Q').text
            except:
                tel = np.NaN
            
            
            # 뽑아온 리스트에 추가
            if adr[0] == '서울':
                temp.extend([name,category,review_num,star,address,tel])
                crawl_list.append(temp)
            else:
                pass
            
            # 다음 가게 클릭을 위해 iframe 이동
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')
            
        # ======= 페이지바 ========
        # 페이지 번호는 최대 5페이지까지 나타남
        # [0] 이전페이지 / [1:5] 페이지번호 / [6] 다음페이지 
        # 페이지바 DOM
        page_bar = driver.find_elements(By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a')
        # 현재페이지
        current_page = driver.find_element(By.CLASS_NAME, 'mBN2s.qxokY')
        # 마지막페이지
        last_num = page_bar[-2].text
        
        # 현재페이지 = 마지막페이지 이면 종료
        if last_num == current_page.text:
            break
        else:
            # 아니면 다음페이지 이동
            page_bar[-1].click()
            

    # 검색상자 검색어 지우기
    driver.switch_to.default_content()
    search_box.clear()
    print(f'{search_id} 약 {page*50}개 수집완료')

# csv 저장
# 파일명 /'w'쓰기모드 / 한글인코딩 / 자동줄바꿈 하지않음
f = open(f'happy_dog_list.csv', 'w', encoding='utf-8-sig', newline='') 
csvwriter = csv.writer(f)

# 컬럼명
csvwriter.writerow(['상호명','업종','리뷰 수','별점','행정구','전화번호'])
# 데이터 입력
csvwriter.writerows(crawl_list)

f.close()